<a href="https://colab.research.google.com/github/lili-hdz/inteligencia_artificial/blob/main/LinkenChamba/codigo_pagina_web.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
Generador de base de datos JSON con 500 candidatos para LinkenChamba
Incluye sistema de puntuación basado en red neuronal multicapa
"""
import json
import random
from faker import Faker
import numpy as np

# Simulación de red neuronal multicapa para calcular puntuaciones
class RedNeuronalPuntuacion:
    """Red neuronal multicapa simple para calcular puntuaciones de candidatos"""
    def __init__(self):
        # Pesos aleatorios inicializados (simulando una red entrenada)
        np.random.seed(42)
        # Capa de entrada: 20 características
        # Capa oculta 1: 15 neuronas
        # Capa oculta 2: 10 neuronas
        # Capa de salida: 5 puestos
        self.W1 = np.random.randn(20, 15) * 0.1
        self.b1 = np.random.randn(15) * 0.1
        self.W2 = np.random.randn(15, 10) * 0.1
        self.b2 = np.random.randn(10) * 0.1
        self.W3 = np.random.randn(10, 5) * 0.1
        self.b3 = np.random.randn(5) * 0.1

    def sigmoid(self, x):
        """Función de activación sigmoide"""
        return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

    def forward(self, X):
        """Propagación hacia adelante"""
        # Normalizar entrada
        X = np.array(X)
        if X.ndim == 1:
            X = X.reshape(1, -1)

        # Capa oculta 1
        Z1 = np.dot(X, self.W1) + self.b1
        A1 = self.sigmoid(Z1)

        # Capa oculta 2
        Z2 = np.dot(A1, self.W2) + self.b2
        A2 = self.sigmoid(Z2)

        # Capa de salida
        Z3 = np.dot(A2, self.W3) + self.b3
        A3 = self.sigmoid(Z3)

        return A3[0] * 100  # Escalar a 0-100%

    def extraer_caracteristicas(self, respuestas):
        """Extrae características numéricas de las respuestas"""
        features = []

        # Habilidades prácticas (10 características binarias)
        habilidades_opciones = [
            "Atender clientes", "Manejar caja", "Organizar archivos",
            "Usar computadora", "Empacar productos", "Limpiar y mantener",
            "Ayudar en preparación", "Tomar mensajes", "Resolver quejas",
            "Ayudar a compañeros"
        ]
        for hab in habilidades_opciones:
            features.append(1 if any(hab.lower() in h.lower() for h in respuestas.get("habilidades_practicas", [])) else 0)

        # Ambiente (5 características one-hot)
        ambiente = respuestas.get("ambiente", "")
        ambientes_opciones = [
            "movimiento", "tranquilo", "interactuar", "concentrado", "adapto"
        ]
        for amb in ambientes_opciones:
            features.append(1 if amb.lower() in ambiente.lower() else 0)

        # Niveles de conocimiento (4 características: 0=Nada, 1=Básico, 2=Intermedio, 3=Avanzado)
        niveles = respuestas.get("niveles", {})
        nivel_map = {"Nada": 0, "Básico": 1, "Intermedio": 2, "Avanzado": 3}
        for tema in ["Productividad", "Organización", "Atención clientes", "Trabajo equipo"]:
            nivel = niveles.get(tema, "Nada")
            features.append(nivel_map.get(nivel, 0) / 3.0)  # Normalizar a 0-1

        # Tipo de trabajo (1 característica)
        tipo_trabajo = respuestas.get("tipo_trabajo", "")
        features.append(1 if "tiempo completo" in tipo_trabajo.lower() else 0.5)

        return features

fake = Faker('es_MX')  # Generador de datos en español

# Definición de puestos y sus criterios de puntuación
PUESTOS = {
    "Atención a Clientes/Ventas": {
        "habilidades_practicas": ["Atender clientes", "Manejar caja", "Resolver quejas", "Tomar mensajes"],
        "herramientas": ["Computadora", "Teléfono", "Caja registradora"],
        "ambiente": ["Interactuar con mucha gente", "Me adapto fácilmente"],
        "actividades": ["Ayudar directamente a clientes"],
        "conocimientos": ["Conceptos básicos de ventas", "Matemáticas", "Lectura rápida"],
        "niveles": {"Atención clientes": ["Intermedio", "Avanzado"], "Productividad": ["Básico", "Intermedio"]},
        "aspiración": ["Trabajo de tiempo completo", "Desarrollarme dentro de esta empresa"],
        "reaccion": ["Pregunto a alguien", "Mantengo la calma"],
        "destaca": ["Ayudar a otros a entenderse", "Motivar al equipo"]
    },
    "Asistente Administrativo": {
        "habilidades_practicas": ["Organizar archivos", "Usar computadora", "Tomar mensajes"],
        "herramientas": ["Computadora", "Teléfono"],
        "ambiente": ["Trabajo tranquilo y organizado", "Trabajo mejor concentrado"],
        "actividades": ["Organizar y poner todo en orden"],
        "conocimientos": ["Lectura rápida", "Escritura clara", "Matemáticas"],
        "niveles": {"Organización": ["Intermedio", "Avanzado"], "Productividad": ["Intermedio", "Avanzado"]},
        "aspiración": ["Trabajo de tiempo completo", "Aprender un oficio"],
        "reaccion": ["Organizo la información", "Mantengo la calma"],
        "destaca": ["Encontrar errores o detalles", "Mantener el orden"]
    },
    "Operario de Producción": {
        "habilidades_practicas": ["Ayudar en preparación", "Empacar productos", "Limpiar y mantener"],
        "herramientas": ["Herramientas básicas", "Equipo de cocina", "Aprendo rápido"],
        "ambiente": ["Me gusta el movimiento", "Me adapto fácilmente"],
        "actividades": ["Crear o arreglar cosas con las manos", "Resolver problemas prácticos"],
        "conocimientos": ["Matemáticas", "Conocimiento de productos específicos"],
        "niveles": {"Productividad": ["Básico", "Intermedio"], "Trabajo equipo": ["Básico", "Intermedio"]},
        "aspiración": ["Trabajo de tiempo completo", "Aprender un oficio"],
        "reaccion": ["Actúo inmediatamente", "Busco la solución yo mismo"],
        "destaca": ["Aprender algo nuevo rápido", "Mantener el orden"]
    },
    "Ayudante General": {
        "habilidades_practicas": ["Empacar productos", "Limpiar y mantener", "Organizar archivos", "Ayudar a compañeros"],
        "herramientas": ["Herramientas básicas", "Aprendo rápido"],
        "ambiente": ["Me adapto fácilmente", "Me gusta el movimiento"],
        "actividades": ["Resolver problemas prácticos", "Aprender cosas nuevas"],
        "conocimientos": ["Aprendo viendo y practicando"],
        "niveles": {"Trabajo equipo": ["Básico", "Intermedio"], "Productividad": ["Básico"]},
        "aspiración": ["Cualquier opción mientras pueda aprender", "Aprender un oficio"],
        "reaccion": ["Pregunto a alguien", "Actúo inmediatamente"],
        "destaca": ["Aprender algo nuevo rápido", "Necesito oportunidad para descubrirlo"]
    },
    "Ayudante en cocina": {
        "habilidades_practicas": ["Ayudar en preparación", "Limpiar y mantener", "Empacar productos"],
        "herramientas": ["Equipo de cocina", "Herramientas básicas"],
        "ambiente": ["Me gusta el movimiento", "Trabajo mejor concentrado"],
        "actividades": ["Crear o arreglar cosas con las manos", "Resolver problemas prácticos"],
        "conocimientos": ["Conocimiento de productos específicos", "Matemáticas"],
        "niveles": {"Trabajo equipo": ["Básico", "Intermedio"], "Productividad": ["Básico", "Intermedio"]},
        "aspiración": ["Trabajo de tiempo completo", "Aprender un oficio"],
        "reaccion": ["Actúo inmediatamente", "Pregunto a alguien"],
        "destaca": ["Aprender algo nuevo rápido", "Mantener el orden"]
    }
}

# Opciones del cuestionario
HABILIDADES_PRACTICAS = [
    "Atender clientes en mostrador o por teléfono",
    "Manejar caja registradora y dar cambio",
    "Organizar archivos o productos en inventario",
    "Usar computadora para escribir documentos básicos",
    "Empacar productos y preparar pedidos",
    "Limpiar y mantener áreas de trabajo ordenadas",
    "Ayudar en preparación de productos o materiales",
    "Tomar mensajes y recados con claridad",
    "Resolver quejas simples de clientes",
    "Ayudar a compañeros con sus tareas cuando es necesario"
]

HERRAMIENTAS = [
    "Computadora (Windows, Internet, email)",
    "Teléfono y sistemas de mensajería",
    "Caja registradora o punto de venta",
    "Herramientas básicas (martillo, destornillador, etc.)",
    "Equipo de cocina o restaurante",
    "Vehículo para entregas",
    "Ninguno de los anteriores, pero aprendo rápido"
]

AMBIENTES = [
    "Me gusta el movimiento y estar activo todo el día",
    "Prefiero trabajo tranquilo y organizado",
    "Disfruto interactuar con mucha gente",
    "Trabajo mejor concentrado en una sola tarea",
    "Me adapto fácilmente a cualquier ambiente"
]

ACTIVIDADES = [
    "Ayudar directamente a clientes o personas",
    "Crear o arreglar cosas con las manos",
    "Organizar y poner todo en orden",
    "Aprender cosas nuevas constantemente",
    "Resolver problemas prácticos inmediatos"
]

CONOCIMIENTOS = [
    "Matemáticas (hacer cuentas, medir, calcular precios)",
    "Lectura rápida y comprensión de instrucciones",
    "Escritura clara para llenar formatos o tomar notas",
    "Conceptos básicos de ventas o servicio al cliente",
    "Conocimiento de productos específicos (comida, ropa, materiales, etc.)",
    "Ninguna en particular, pero aprendo viendo y practicando"
]

NIVELES = ["Nada", "Básico", "Intermedio", "Avanzado"]

TIPOS_TRABAJO = [
    "Trabajo de medio tiempo (4-6 horas diarias)",
    "Trabajo de tiempo completo (8 horas)",
    "Trabajo por proyectos o temporal",
    "Aprendizaje o capacitación con salario básico",
    "Cualquier opción mientras pueda aprender y crecer"
]

LOGROS = [
    "Aprender un oficio o skill específico",
    "Tener un ingreso estable para mis gastos",
    "Ganar experiencia para mejor empleo después",
    "Desarrollarme dentro de esta empresa",
    "Descubrir en qué soy bueno profesionalmente"
]

REACCIONES = [
    "Pregunto a alguien con más experiencia",
    "Busco la solución yo mismo investigando",
    "Organizo la información para entender mejor",
    "Actúo inmediatamente con lo que sé",
    "Mantengo la calma y evalúo opciones"
]

DESTACA = [
    "Cuando hay que ayudar a otros a entenderse",
    "Cuando hay que encontrar errores o detalles",
    "Cuando hay que aprender algo nuevo rápido",
    "Cuando hay que mantener el orden en el caos",
    "Cuando hay que motivar al equipo",
    "No estoy seguro, necesito oportunidad para descubrirlo"
]

MOTIVACIONES = [
    "Necesidad económica inmediata",
    "Aprender y desarrollar habilidades",
    "Estabilidad y crecimiento a largo plazo",
    "Ambiente de trabajo positivo",
    "Todas las anteriores"
]

# Instancia global de la red neuronal
_red_neuronal = None

def obtener_red_neuronal():
    """Obtiene o crea la instancia de la red neuronal"""
    global _red_neuronal
    if _red_neuronal is None:
        _red_neuronal = RedNeuronalPuntuacion()
    return _red_neuronal

def calcular_puntuacion_puesto(respuestas, puesto):
    """Calcula la puntuación usando red neuronal multicapa"""
    red_neuronal = obtener_red_neuronal()

    # Extraer características
    features = red_neuronal.extraer_caracteristicas(respuestas)

    # Obtener todas las puntuaciones de la red neuronal
    puntuaciones_todas = red_neuronal.forward(features)

    # Mapear a puestos específicos
    indice_puesto = list(PUESTOS.keys()).index(puesto)
    puntuacion = puntuaciones_todas[indice_puesto]

    # Ajustar basado en criterios específicos del puesto
    criterios = PUESTOS[puesto]
    ajuste = 0

    # Ajuste por habilidades específicas
    habilidades_seleccionadas = respuestas.get("habilidades_practicas", [])
    matches = sum(1 for hab in habilidades_seleccionadas
                  if any(crit.lower() in hab.lower() for crit in criterios["habilidades_practicas"]))
    ajuste += matches * 3

    # Ajuste por ambiente
    ambiente = respuestas.get("ambiente", "")
    if any(crit.lower() in ambiente.lower() for crit in criterios["ambiente"]):
        ajuste += 5

    # Normalizar y limitar
    puntuacion_final = min(100, max(0, puntuacion + ajuste))
    return round(puntuacion_final, 2)

def generar_respuestas_aleatorias():
    """Genera respuestas aleatorias al cuestionario"""
    return {
        "habilidades_practicas": random.sample(HABILIDADES_PRACTICAS, k=random.randint(3, 5)),
        "herramientas": random.sample(HERRAMIENTAS, k=random.randint(2, 4)),
        "ambiente": random.choice(AMBIENTES),
        "actividades": random.sample(ACTIVIDADES, k=random.randint(1, 3)),
        "conocimientos": random.sample(CONOCIMIENTOS, k=random.randint(2, 4)),
        "niveles": {
            "Productividad": random.choice(NIVELES),
            "Organización": random.choice(NIVELES),
            "Atención clientes": random.choice(NIVELES),
            "Trabajo equipo": random.choice(NIVELES)
        },
        "tipo_trabajo": random.choice(TIPOS_TRABAJO),
        "logros": random.sample(LOGROS, k=random.randint(1, 3)),
        "reaccion": random.choice(REACCIONES),
        "destaca": random.sample(DESTACA, k=random.randint(1, 3)),
        "motivacion": random.choice(MOTIVACIONES),
        "unico": fake.text(max_nb_chars=200)
    }

def generar_candidato():
    """Genera un candidato completo con datos y respuestas"""
    respuestas = generar_respuestas_aleatorias()

    # Calcular puntuaciones para cada puesto
    puntuaciones = {}
    for puesto in PUESTOS.keys():
        puntuaciones[puesto] = calcular_puntuacion_puesto(respuestas, puesto)

    candidato = {
        "id": fake.uuid4(),
        "nombre": fake.name(),
        "email": fake.email(),
        "telefono": fake.phone_number(),
        "direccion": fake.address().replace('\n', ', '),
        "respuestas_cuestionario": respuestas,
        "puntuaciones": puntuaciones,
        "fecha_registro": fake.date_between(start_date='-1y', end_date='today').isoformat()
    }

    return candidato

def main():
    """Genera la base de datos con 500 candidatos usando red neuronal"""
    print("Inicializando red neuronal multicapa...")
    print("Generando base de datos con 500 candidatos...")
    candidatos = []

    for i in range(500):
        candidato = generar_candidato()
        candidatos.append(candidato)
        if (i + 1) % 50 == 0:
            print(f"   Generados {i + 1}/500 candidatos...")

    base_datos = {
        "version": "1.0",
        "total_candidatos": len(candidatos),
        "puestos_disponibles": list(PUESTOS.keys()),
        "metodo_puntuacion": "Red neuronal multicapa (3 capas: 20->15->10->5)",
        "candidatos": candidatos
    }

    # Guardar en JSON
    with open("base_datos_candidatos.json", "w", encoding="utf-8") as f:
        json.dump(base_datos, f, ensure_ascii=False, indent=2)

    print(f"\n[OK] Base de datos generada exitosamente!")
    print(f"   Archivo: base_datos_candidatos.json")
    print(f"   Total de candidatos: {len(candidatos)}")
    print(f"   Metodo: Red neuronal multicapa")

    # Mostrar estadísticas
    print("\nEstadisticas de puntuaciones promedio (calculadas con red neuronal):")
    for puesto in PUESTOS.keys():
        puntuaciones = [c["puntuaciones"][puesto] for c in candidatos]
        promedio = np.mean(puntuaciones)
        maximo = np.max(puntuaciones)
        minimo = np.min(puntuaciones)
        print(f"   {puesto}:")
        print(f"      Promedio: {promedio:.2f}%")
        print(f"      Maximo: {maximo:.2f}%")
        print(f"      Minimo: {minimo:.2f}%")

if __name__ == "__main__":
    main()

